• Create a Gradio app using LangChain with the following features:
1. SQL Query Generation: Generate SQL queries from natural language.
2. SQL Safety Check: Implement basic safety checks for SQL queries to prevent
malicious input.
3. Conversational Memory: Add multi-round conversational capabilities with
a chat box and history.
4. Question-Answering System: Build a complete question-answering system
for the course schedule.

• The Gradio app must include:
* A conversation box that displays the full chat history.
* The ability to handle multi-round interactions for context-aware querying.

#### Change the column names to make it easire to understand

In [1]:
import pandas as pd
import sqlite3

# Load the data from the database
query = "SELECT * FROM courses"
conn = sqlite3.connect("courses.db")

df = pd.read_sql(query, conn)

# Rename columns to be more intuitive
df = df.rename(columns={
    "Mon": "Monday_start",
    "MonTo": "Monday_end",
    "Tue": "Tuesday_start",
    "TueTo": "Tuesday_end",
    "Wed": "Wednesday_start",
    "WedTo": "Wednesday_end",
    "Thu": "Thursday_start",
    "ThuTo": "Thursday_end",
    "Fri": "Friday_start",
    "FriTo": "Friday_end",
    "Sat": "Saturday_start",
    "SatTo": "Saturday_end"
})

# Save the modified DataFrame back to the database
df.to_sql("courses", conn, if_exists="replace", index=False)


135

#### Set up models

In [2]:
# Configure LLM options
from langchain_ollama import ChatOllama
import os
from langchain_community.llms import DeepInfra
from langchain_openai import ChatOpenAI

# Local LLM setup
local_model = ChatOllama(model="qwen2.5:7b")
# local_model = ChatOllama(model = "phi3.5:3.8b-mini-instruct-q8_0")

# API setup (optional)
os.environ["DEEPINFRA_API_TOKEN"] = str(os.environ.get("DEEPINFRA_API_KEY"))
os.environ["OPENAI_API_KEY"] = str(os.environ.get("OPENAI_API_KEY"))
# deepinfra = DeepInfra(model_id="meta-llama/Meta-Llama-3.1-70B-Instruct")

# api_model = None ### 
# https://api.python.langchain.com/en/latest/llms/langchain_community.llms.deepinfra.DeepInfra.html

# Choose which model to use
# llm = DeepInfra(model_id="meta-llama/Meta-Llama-3.1-70B-Instruct")  # Switch to api_model if using Replicate
llm = local_model  # Switch to api_model if using Replicate
llm = ChatOpenAI(model="gpt-4o-mini")

#### Prepare some functions that would be used later

In [3]:
from langchain_community.utilities import SQLDatabase

# Initialize database connection
db = SQLDatabase.from_uri("sqlite:///courses.db", sample_rows_in_table_info=0)

# Helper functions
def get_schema(_):
    """Retrieve database schema"""
    return db.get_table_info()

def run_query(query):
    """Execute SQL query"""
    return db.run(query)

In [4]:
def parse(string):
    """To parse the responce of llm, get the pure SQL out from ```sql <SQL>```"""
    
    if "```" in string:
        string = string.split("```")
        string = string[1].replace("sql","")
        
    return string

#### Prepare info needed for prompt

In [ ]:
import json

# Load the data from the database into a pandas DataFrame
df = pd.read_sql("SELECT * FROM courses", conn)

# Select 3 keys from the dictionary: 'Instructor', 'Department', 'program'
column_unique_values = {col: df[col].unique().tolist() for col in df.columns}
selected_keys = ['Instructor', 'Department', 'Program']
selected_values = {key: column_unique_values[key] for key in selected_keys}

# # Create a dictionary with column names as keys and unique values as values
# column_unique_values = {col: df[col].unique().tolist() for col in df.columns}
# print(column_unique_values)

# # Get all keys of the dictionary
# keys = list(column_unique_values.keys())
# print(keys)

# column_unique_values_str = json.dumps(column_unique_values)
column_unique_values_str = json.dumps(selected_values)

print(column_unique_values_str)

{"Instructor": ["Michael Zhao", "Kevin Ren", "Qihu Zhang", "Zhanglin Cui", "Varies", "TBD", "J. Rawls", "M. Sanchez", "F. Yuan", "M. Torley", "Vivian Liu", "Bill Tse", "Ethan/Lilian Li", "Lilian Li/S. Zhu", "Lilian Li", "Dan Pelonis", "Dan Pelonis &Zhen Su", "Eric Bess", "Penny", "CL& Penny", "TBD/Anh Cao", "T. Liu", "M. Xu", "J. Shi", "X. Tan", "Zeno Yu", "Sand Y. Zhu", "Zheng Qu", "Miao Yu", "T. Harrison", "T. Liu/Anh Cao", "B. McDuffie", "S. Grodnick", "H. Jiang/TBD", "J. Wang", "X. Wang", "V. Liu", "G. Shang", "A. Chen", "S. Jiang", "A. Chen/S. Jiang", "S. Jiang (B. Du - TA)", "M. Liu", "Y. Tsuai", "S. Yang", "Y. Chen", "A. Chen/Y. Tsuai", "A.Chen"], "Department": ["Applied Math & Statistics", "ART", "Biomedical Science (BMS)", "Computer Science (CS), Computer Networks and Cybersecurity (CNCS)", "Data Science (DS)", "General Education (GE)", "Dance "], "Program": ["(Bachelor of Science)", "(Master of Science)", "Arts Management (BFA)", "Graphic Design Concentration (BFA in Fine Art

#### SQL from natural language

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# SQL generation prompt
sql_prompt_template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""

system_prompt = """Your are an helpful assistence that helps answer questions about course on Spring Semeter of 2025. \
                All needed information is stored in course.db
                Here is a dictionary storing something values of the database. 
                The key of dictionary is Column name, and the value is a list of values of the database under the corresponding Column name
                
                Dictionary : \n
                """+column_unique_values_str[1:-2]

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
    ("human", sql_prompt_template),
])

# Build query generation chain
sql_generator = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
    | parse
)

In [7]:
question = "How many courses does Mr. Micheal Zhao teaches"
# question = " What are avaliale Departments?"
# question = "Which courses are available in the Biomedical sciemce department for Spring 2025?"
sql = sql_generator.invoke({"question": question})
print(str(sql))


SELECT COUNT(*) 
FROM courses 
WHERE Instructor = 'Michael Zhao';



#### Natural Language response

In [8]:
# Natural language response generation, interpret sql
response_template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""

nl_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given an input question and SQL response, convert it to a natural language answer. No pre-amble."),
    ("human", response_template),
])

from langchain_core.runnables import RunnableLambda

def print_func(x):
    print(x['query'])
    return x

printfunc = RunnableLambda(print_func) # object
# def printfunc(input):
#     print(input["query"])
#     return input

# Complete chain with natural language output
complete_chain = (
    RunnablePassthrough.assign(query=sql_generator)
    | printfunc
    | RunnablePassthrough.assign( # dictionary
        schema=get_schema,
        response=lambda x: db.run(x["query"]), # db.run exsecute database, x is the input
    )
    | nl_prompt
    | llm
)

In [9]:
# question = "How many courses does Mr. Micheal Zhao teaches"
# question = " What are avaliale Departments?"
question = "Which courses are available in the Biomedical sciemce department for Spring 2025?"
nl_response = complete_chain.invoke({"question": question})
print(nl_response)


SELECT * FROM courses WHERE Department = 'Biomedical Science (BMS)';

content='The available courses in the Biomedical Science department for Spring 2025 are:\n\n1. BMS343 - Biomedical Ethics, 2 credits, taught by Anh Cao.\n2. BSC102 - General Biology II, 3 credits, taught by T. Liu.\n3. BSC102L - General Biology II Lab, 1 credit, taught by T. Liu.\n4. CHM102 - General Chemistry II, 3 credits, taught by M. Xu.\n5. CHM102L - General Chemistry II Lab, 1 credit, taught by M. Xu.\n6. BSC212 - Anatomy and Physiology II, 3 credits, taught by T. Liu.\n7. BSC212L - Anatomy and Physiology II Lab, 1 credit, taught by T. Liu.\n8. CHM212 - Organic Chemistry II (Directed Study, online), 3 credits, taught by J. Shi.\n9. CHM212L - Organic Chemistry II Lab (Directed Study, online), 1 credit, taught by J. Shi.\n10. BSC322 - Biochemistry, 3 credits, taught by X. Tan.\n11. BMS141 - Introduction to US Healthcare (online), 2 credits, taught by TBD.\n12. BSC432 - Introduction to Pharmacology (Directed Stud

In [12]:
from validate_safty import SafeQueryExecutor

# print(nl_response.get('query'))
# Initialize the safe executor
safe_executor = SafeQueryExecutor(db_connection=db, query_chain=sql_generator, llm_chain=complete_chain)

# Example queries
def test_queries():
    # Valid query
    # result1 = safe_executor.execute_safe_query("What is salary of Klay Thompson?")
    result1 = safe_executor.execute_safe_query("Which courses are available in the Biomedical sciemce department for Spring 2025?")
    print("\nValid Query Result:")
    print(result1)
    
    # # Invalid query with SQL injection attempt
    # result2 = safe_executor.execute_safe_query("SELECT * FROM courses; DROP TABLE courses;")
    # print("\nInvalid Query Result:")
    # print(result2)
    
    # # Query with restricted words
    # result3 = safe_executor.execute_safe_query("delete all players from Warriors")
    # print("\nRestricted Query Result:")
    # print(result3)

# Run test queries
test_queries()


Valid Query Result:
{'success': False, 'error': ['Multiple SQL statements are not allowed'], 'query': "\nSELECT * FROM courses WHERE Department = 'Biomedical Science (BMS)';\n", 'result': None}
